In [ ]:
my_token = '6801ebee-4378-4cea-a1b2-bc84e115bbbbc8a2aed2-c20d-442f-a162-2ed391facfca'
DEBUG_LOGGING = False

In [63]:
import requests
import queue
import json
import time
import math
from enum import IntEnum


In [91]:
CELL = 160
HALF_CELL = CELL / 2
SENSING_OFFSET = 40
ROTATION_SPEED = 220
FORWARD_SPEED = 70
FRONT_REFERENCE = 70
ANGLE_OFFSET = 10
KP_ROT = 4.0
KP_STEERING = 0.4
MAX_SPEED = 200
PRE_TURN_SPEED = 100

LEFT_REFERENCE = 59
RIGHT_REFERENCE = 49

WALL_THRESHOLD = 130
FRONT_WALL_THRESHOLD = 180
PRE_TURN_THRESHOLD = 110
CENTER_REFERENCE = 80


class Mouse:
    
    
    
    def __init__(self, token: str, manual_control: bool = False):
        super().__init__()
        self._api_route = 'http://127.0.0.1:8801/api/v1'
        self._token = token
        self.left_wall_distance = 0
        self.front_wall_distance = 0
        self.right_wall_distance = 0
        self.left_wall = False
        self.front_wall = False
        self.right_wall = False
        
        self.angle = 0
        self.prev_angle = 0
        self.reference_angle = 0
        self.offset_x = 0
        self.offset_y = 0
        self.pos = 0
        self.speed = 0
        self.max_speed = MAX_SPEED
        self.manual_control = manual_control
        
        self.rot_error = 0
        self._steering_enabled = True
        
        self.update_sensor_data(init_update=True)
        self.reference_angle = self.angle
        self.prev_angle = self.angle
        
        
        
    def _make_action(self, action: str):
        requests.post(f'{self._api_route}/robot-cells/{action}?token={self._token}')
        time.sleep(0.1)
        
    def _move_motors(self, pwm_left: int | float, pwm_right: int | float, m_time: float = 0.1):
        pwm_left = int(pwm_left)
        pwm_right = int(pwm_right)
        requests.post(f'{self._api_route}/robot-motors/move?token={self._token}&l={pwm_left}&l_time={m_time}&r={pwm_right}&r_time={m_time}')
        time.sleep(m_time)
    
    def _calc_dist(self, start_y: float, start_x: float):
        return math.sqrt((start_y - self.offset_y) ** 2 + (start_x - self.offset_x) ** 2)
    
    def forward(self):
        if self.manual_control:
            pass
        else:
            self._make_action('forward')
            
    def move(self, dist: float, stop_threshold: float = PRE_TURN_THRESHOLD):
        self.update_sensor_data()
        cur_y = self.offset_y
        cur_x = self.offset_x
        cur_dist = 0
        while cur_dist < dist:
            if self.speed < self.max_speed:
                left_pwm = FORWARD_SPEED + self.rot_error
                right_pwm = FORWARD_SPEED - self.rot_error
                self._move_motors(left_pwm, right_pwm)
            else:
                time.sleep(0.1)
            self.update_sensor_data()
            diff = self._calc_dist(cur_y, cur_x)
            self.speed = diff / 0.1
            cur_dist += diff
            self.pos += diff
            cur_y = self.offset_y
            cur_x = self.offset_x
            if self.front_wall_distance < stop_threshold:
                return 
    
    def wait_until_position(self, pos: float, stop_threshold: float = PRE_TURN_THRESHOLD):
        self.update_sensor_data()
        cur_y = self.offset_y
        cur_x = self.offset_x
        while self.pos < pos:
            if self.speed < self.max_speed:
                left_pwm = FORWARD_SPEED + self.rot_error
                right_pwm = FORWARD_SPEED - self.rot_error
                self._move_motors(left_pwm, right_pwm)
            else:
                time.sleep(0.1)
            self.update_sensor_data()
            diff = self._calc_dist(cur_y, cur_x)
            self.speed = diff / 0.1
            self.pos += diff
            cur_y = self.offset_y
            cur_x = self.offset_x
            if self.front_wall_distance < stop_threshold:
                return 
            
    def stop(self):
        self.update_sensor_data()
        cur_y = self.offset_y
        cur_x = self.offset_x
        while self.speed > 5:
            self._move_motors(-FORWARD_SPEED, -FORWARD_SPEED)
            self.update_sensor_data()
            diff = self._calc_dist(cur_y, cur_x)
            self.speed = diff / 0.1
            self.pos += diff
            cur_y = self.offset_y
            cur_x = self.offset_x
    
    def left(self):
        if self.manual_control:
            self._steering_enabled = False
            self.max_speed = PRE_TURN_SPEED
            self.wait_until_position(CELL)
            self.reference_angle -= 90
            self.update_sensor_data()
            while self.angle > self.reference_angle + ANGLE_OFFSET:
                self._move_motors(-ROTATION_SPEED, ROTATION_SPEED)
                self.update_sensor_data()
            
            self.max_speed = MAX_SPEED
            self.move(40)
            self.pos = CELL
            self._steering_enabled = True
        else:
            self._make_action('left')

    def right(self):
        if self.manual_control:
            self._steering_enabled = False
            self.max_speed = PRE_TURN_SPEED
            self.wait_until_position(CELL)
            self.reference_angle += 90
            self.update_sensor_data()
            while self.angle < self.reference_angle - ANGLE_OFFSET:
                self._move_motors(ROTATION_SPEED, -ROTATION_SPEED)
                self.update_sensor_data()
            
            self.max_speed = MAX_SPEED
            self.move(40)
            self.pos = CELL
            self._steering_enabled = True
        else:
            self._make_action('right')
            
    def right_in_place(self):
        self._steering_enabled = False
        self.reference_angle += 90
        self.update_sensor_data()
        while self.angle < self.reference_angle - ANGLE_OFFSET:
            self._move_motors(ROTATION_SPEED, -ROTATION_SPEED)
            self.update_sensor_data()
        
        self._steering_enabled = True
        
    def left_in_place(self):
        self._steering_enabled = False
        self.reference_angle -= 90
        self.update_sensor_data()
        while self.angle > self.reference_angle + ANGLE_OFFSET:
            self._move_motors(-ROTATION_SPEED, ROTATION_SPEED)
            self.update_sensor_data()
        
        self._steering_enabled = True
        
    def around(self):
        self.left_in_place()
        self.left_in_place()
        
    def get_sensors(self):
        return requests.get(f'{self._api_route}/robot-cells/sensor-data?token={self._token}').json()
    
    def _calc_errors(self):
        ang_error = (self.reference_angle - self.angle) * KP_ROT
        
        pos_error = 0
        left_err = (self.left_wall_distance - LEFT_REFERENCE)
        right_err = (self.right_wall_distance - RIGHT_REFERENCE)
        if self.left_wall and self.right_wall:
            pos_error = left_err - right_err
        elif self.left_wall:
            pos_error = 2 * left_err
        elif self.right_wall:
            pos_error = 2 * right_err
        pos_error *= KP_STEERING
        
        if not self._steering_enabled:
            pos_error = 0
        
        self.rot_error = ang_error - pos_error
    
    def update_sensor_data(self, init_update: bool = False):
        sensor_data = self.get_sensors()
        self.front_wall_distance = sensor_data['front_distance']
        if self.manual_control and not init_update:
            self.left_wall_distance = sensor_data['left_45_distance']
            self.right_wall_distance = sensor_data['right_45_distance']
        else:
            self.left_wall_distance = sensor_data['left_side_distance']
            self.right_wall_distance = sensor_data['right_side_distance']
        self.left_wall = self.left_wall_distance < WALL_THRESHOLD
        self.front_wall = self.front_wall_distance < FRONT_WALL_THRESHOLD
        self.right_wall = self.right_wall_distance < WALL_THRESHOLD
        
        angle = sensor_data['rotation_yaw']
        if angle < -90:
            if self.prev_angle > 90:
                self.prev_angle -= 360
        if angle > 90:
            if self.prev_angle < -90:
                self.prev_angle += 360
        diff = angle - self.prev_angle
        self.angle += diff
        self.prev_angle = angle
        
        self.offset_x = sensor_data['down_x_offset']
        self.offset_y = sensor_data['down_y_offset']
        
        self._calc_errors()
        print(self)
    
    def __str__(self):
        return (
            f'< {self.left_wall_distance} {self.left_wall} \n'
            f'^ {self.front_wall_distance} {self.front_wall} \n'
            f'> {self.right_wall_distance} {self.right_wall} \n'
            f'angle: {self.angle} ({self.reference_angle})\n'
            f'x: {self.offset_x} y: {self.offset_y}\n'
            f'error: {self.rot_error}'
        )
    
    
    

In [92]:
class Directions(IntEnum):
    up = 0
    right = 1
    down = 2
    left = 3
    
VISITED = 0xF0
UP_WALL_VISITED = 0b10000000
RIGHT_WALL_VISITED = 0b01000000
DOWN_WALL_VISITED = 0b00100000
LEFT_WALL_VISITED = 0b00010000
UP_WALL = 0b00001000
RIGHT_WALL = 0b00000100
DOWN_WALL = 0b00000010
LEFT_WALL = 0b00000001

WALLS_VISITED = (UP_WALL_VISITED, RIGHT_WALL_VISITED, DOWN_WALL_VISITED, LEFT_WALL_VISITED)
WALLS = (UP_WALL, RIGHT_WALL, DOWN_WALL, LEFT_WALL)
DIRECTION_TO_CHAR = ('^', '>', 'v', '<')

RESULT_WALLS_MAPPING = {
    0: 0,
    LEFT_WALL: 1,
    UP_WALL: 2,
    RIGHT_WALL: 3,
    DOWN_WALL: 4,
    LEFT_WALL | DOWN_WALL: 5,
    RIGHT_WALL | DOWN_WALL: 6,
    UP_WALL | RIGHT_WALL: 7,
    UP_WALL | LEFT_WALL: 8,
    LEFT_WALL | RIGHT_WALL: 9,
    UP_WALL | DOWN_WALL: 10,
    UP_WALL | RIGHT_WALL | DOWN_WALL: 11,
    LEFT_WALL | UP_WALL | RIGHT_WALL: 12,
    UP_WALL | DOWN_WALL | LEFT_WALL: 13,
    RIGHT_WALL | DOWN_WALL | LEFT_WALL: 14,
    UP_WALL | RIGHT_WALL | DOWN_WALL | LEFT_WALL: 15, 
}

class Maze:
    shape = 16
    start = (15, 0)
    start_direction = Directions.up
    finish = [(7, 7), (7, 8), (8, 7), (8, 8)]
    
    
    NEIGHBOURS = (
        (shape - 1, 0),
        (0, 1),
        (1, 0),
        (0, shape - 1),
    )
    
    def __init__(self, token: str):
        self._api_route = 'http://127.0.0.1:8801/api/v1'
        self._token = token
        self.mouse_position = self.start
        self.mouse_direction = self.start_direction
        self.maze = []
        self.walls = []
        self.path = []
        self.reset_maze()
    
    @property
    def cur_x(self):
        return self.mouse_position[1]
    
    @property
    def cur_y(self):
        return self.mouse_position[0]
    
    @property
    def finish_cell(self):
        return self.finish[0]
        
    def reset_maze(self):
        self.maze = [[0 for _ in range(self.shape)] for _ in range(self.shape)]
        self.walls = [[0 for _ in range(self.shape)] for _ in range(self.shape)]
        
        for i in range(self.shape):
            self.walls[i][0] |= LEFT_WALL | LEFT_WALL_VISITED
            self.walls[i][self.shape - 1] |= RIGHT_WALL | RIGHT_WALL_VISITED
            self.walls[0][i] |= UP_WALL | UP_WALL_VISITED
            self.walls[self.shape - 1][i] |= DOWN_WALL | DOWN_WALL_VISITED
            
    def reset_position(self):
        self.mouse_position = self.start
        self.mouse_direction = self.start_direction
        requests.post(f'{self._api_route}/maze/restart?token={self._token}').json()
        
    def is_visited(self, position: tuple | None = None):
        if position is None:
            position = self.mouse_position
        return (self.walls[position[0]][position[1]] & VISITED) == VISITED
        
    def set_walls(self, is_left_wall: bool, is_front_wall: bool, is_right_wall: bool):
        if self.is_visited():
            return
        left_wall_idx = (self.mouse_direction + 3) % 4
        front_wall_idx = self.mouse_direction
        right_wall_idx = (self.mouse_direction + 1) % 4

        _walls = [is_left_wall, is_front_wall, is_right_wall]
        _walls_idx = [left_wall_idx, front_wall_idx, right_wall_idx]

        for i in range(len(_walls)):
            wall_idx = _walls_idx[i]
            neigh_y = (self.cur_y + self.NEIGHBOURS[wall_idx][0]) % self.shape
            neigh_x = (self.cur_x + self.NEIGHBOURS[wall_idx][1]) % self.shape
            neigh_wall_idx = (wall_idx + 2) % 4
            
            if _walls[i] and not (self.walls[self.cur_y][self.cur_x] & WALLS_VISITED[wall_idx]):
                self.walls[self.cur_y][self.cur_x] |= WALLS[wall_idx]
                self.walls[neigh_y][neigh_x] |= WALLS[neigh_wall_idx]
            
            self.walls[self.cur_y][self.cur_x] |= WALLS_VISITED[wall_idx]
            self.walls[neigh_y][neigh_x] |= WALLS_VISITED[neigh_wall_idx]
    
    def get_wall(self, direction: Directions) -> bool:
        _dir = (self.mouse_direction + direction) % 4
        return self.walls[self.cur_y][self.cur_x] & WALLS[_dir] == WALLS[_dir]
    
    def update_direction(self, change: Directions):
        self.mouse_direction = (self.mouse_direction + change) % 4

    def update_position(self):
        new_y = (self.cur_y + self.NEIGHBOURS[self.mouse_direction][0]) % self.shape
        new_x = (self.cur_x + self.NEIGHBOURS[self.mouse_direction][1]) % self.shape
        self.mouse_position = (new_y, new_x)
    
    def check_wall(self, position: tuple, wall: int):
        return (self.walls[position[0]][position[1]] & wall) == wall
    
    def get_neighbours(self, position: tuple | None = None) -> list[dict]:
        if position is None:
            position = self.mouse_position
        neighbours = []
        for i in range(4):
            neigh_y = (position[0] + self.NEIGHBOURS[i][0]) % self.shape
            neigh_x = (position[1] + self.NEIGHBOURS[i][1]) % self.shape
            neigh_val = self.maze[neigh_y][neigh_x]
            if not self.check_wall(position, WALLS[i]):
                neigh_data = {
                    'dir': i,
                    'pos': (neigh_y, neigh_x),
                    'val': neigh_val
                }
                neighbours.append(neigh_data)
        return neighbours
    
    def floodfill(self):
        for y in range(self.shape):
            for x in range(self.shape):
                self.maze[y][x] = 0

        visited = [[0 for _ in range(self.shape)] for _ in range(self.shape)]

        visited[self.finish_cell[0]][self.finish_cell[1]] = 1

        to_process = queue.Queue()
        to_process.put(self.finish_cell)
        while not to_process.empty():
            current_pos = to_process.get()
            neighbours = self.get_neighbours(position=current_pos)
            for neigh in neighbours:
                neigh_y, neigh_x = neigh['pos']
                if not visited[neigh_y][neigh_x]:
                    visited[neigh_y][neigh_x] = 1
                    to_process.put((neigh_y, neigh_x))
                    self.maze[neigh_y][neigh_x] = self.maze[current_pos[0]][current_pos[1]] + 1
                    
    def find_path(self, start: tuple) -> bool:
        self.path = []
        cur_val = self.maze[start[0]][start[1]]
        current_pos = start
        direction = self.mouse_direction
    
        while cur_val != 0:
            neighbours = self.get_neighbours(current_pos)
            for neigh in neighbours:
                neigh_y, neigh_x = neigh['pos']
                neigh_val = neigh['val']
                neigh_dir = neigh['dir']
    
                if (cur_val - 1) == neigh_val:
                    cur_val = neigh_val
                    current_pos = (neigh_y, neigh_x)
                    if direction == neigh_dir:
                        self.path.append('F')
                    elif (direction + 1) % 4 == neigh_dir:
                        self.path.append('R')
                        self.path.append('F')
                    elif (direction + 2) % 4 == neigh_dir:
                        self.path.append('A')
                        self.path.append('F')
                    elif (direction + 3) % 4 == neigh_dir:
                        self.path.append('L')
                        self.path.append('F')
                    direction = neigh_dir
                    break
        return len(self.path) > 0
    
    def get_next_move(self) -> str:
        return self.path.pop(0)
    
    def path_not_empty(self):
        return len(self.path) > 0
    
    def on_finish(self):
        return self.mouse_position in self.finish
    
    def lock_maze(self):
        for y in range(self.shape):
            for x in range(self.shape):
                for i, wall in enumerate(WALLS_VISITED):
                    if (self.walls[y][x] & wall) == 0:
                        self.walls[y][x] |= WALLS[i]
                        
    def save_maze(self):
        with open('maze.json', 'w') as f:
            json.dump(self.walls, f)
    
    def print_maze(self, show_weights: bool = True):
        if not DEBUG_LOGGING:
            return
        cell_shape = 3
        # print header
        for x in range(self.shape):
            print('+', end='')
            for sub_x in range(cell_shape):
                print('-', end='')
        print('+')
        for y in range(self.shape):
            for sub_y in range(cell_shape - 1):
                for x in range(self.shape):
                    if show_weights:
                        maze_value = self.maze[y][x]
                    else:
                        maze_value = int(self.is_visited((y, x)))
                    wall_value = self.walls[y][x]
                    left_wall = wall_value & LEFT_WALL
                    down_wall = wall_value & DOWN_WALL
                    
                    if sub_y == 0:
                        if left_wall:
                            print('|', end='')
                        else:
                            print(' ', end='')
                        if y == self.cur_y and x == self.cur_x:
                            print(' ', end='')
                            print(DIRECTION_TO_CHAR[self.mouse_direction], end='')
                            print(' ', end='')
                        elif maze_value > 99:
                            print(maze_value, end='')
                        elif maze_value > 9:
                            print(' ', end='')
                            print(maze_value, end='')
                        else:
                            print(' ', end='')
                            print(maze_value, end='')
                            print(' ', end='')
                    else:
                        print('+', end='')
                        if down_wall:
                            print('---', end='')
                        else:
                            print('   ', end='')
                if sub_y == 1:
                    print('+')
                else:
                    print('|')
                    
    def post_result(self):
        result = [[0 for _ in range(self.shape)] for _ in range(self.shape)]
        mask = UP_WALL | RIGHT_WALL | DOWN_WALL | LEFT_WALL
        for y in range(self.shape):
            for x in range(self.shape):
                result[y][x] = RESULT_WALLS_MAPPING[self.walls[y][x] & mask]
        return requests.post(f'{self._api_route}/matrix/send?token={self._token}', json=result).json()
    

In [93]:
class Solver:
    def __init__(self, token: str, manual_control: bool = False):
        self.mouse = Mouse(token, manual_control)
        self.maze = Maze(token)
        if manual_control:
            self.mouse.pos = CELL + HALF_CELL
        self.manual_control = True
        
    def _scan_position(self):
        self.mouse.update_sensor_data()
        self.maze.set_walls(self.mouse.left_wall, self.mouse.front_wall, self.mouse.right_wall)
    
    def _move_by_direction(self, direction: Directions):
        direction = direction % 4
        current_direction = self.maze.mouse_direction
        if current_direction == direction:
            print('front')
            if self.manual_control:
                self.mouse.pos -= CELL
                self.mouse.wait_until_position(CELL - SENSING_OFFSET)
        elif (current_direction + 1) % 4 == direction:
            print('right')
            self.mouse.right()
            self.maze.update_direction(Directions.right)
        elif (current_direction + 2) % 4 == direction:
            print('around')
            self.mouse.pos -= CELL
            self.mouse.wait_until_position(HALF_CELL - SENSING_OFFSET, stop_threshold=CENTER_REFERENCE)
            self.mouse.stop()
            self.mouse.around()
            self.maze.update_direction(Directions.down)
        elif (current_direction + 3) % 4 == direction:
            print('left')
            self.mouse.left()
            self.maze.update_direction(Directions.left)
        if not self.manual_control:
            self.mouse.forward()
        self.maze.update_position()
        
    def left_hand(self):
        while self.maze.visited_count < 256:
            self._scan_position()
            self.maze.print_maze()
            current_direction = self.maze.mouse_direction
            if not self.mouse.left_wall:
                self._move_by_direction(current_direction + 3)
            elif not self.mouse.front_wall:
                self._move_by_direction(current_direction)
            elif not self.mouse.right_wall:
                self._move_by_direction(current_direction + 1)
            else:
                self._move_by_direction(current_direction + 2)
    
    def dfs(self):
        self._scan_position()
        self.maze.print_maze(show_weights=False)
        for neighbour in self.maze.get_neighbours():
            neigh_pos = neighbour['pos']
            if not self.maze.is_visited(neigh_pos):
                neigh_direction = neighbour['dir']
                # move to neighbour cell
                self._move_by_direction(neigh_direction)
                # find new path
                self.dfs()
                # return to the previous cell using prev_direction
                self._move_by_direction(neigh_direction + 2)
                
    def shortest(self):
        self.maze.set_walls(self.mouse.left_wall, self.mouse.front_wall, self.mouse.right_wall)
        self.maze.floodfill()
        if self.maze.get_wall(Directions.up):
            self.mouse.right_in_place()
            self.maze.update_direction(Directions.right)
            
        path_exists = self.maze.find_path(self.maze.mouse_position)
        self.maze.print_maze()
        recalculate = False
    
        if path_exists:
            while path_exists and not self.maze.on_finish():
                while self.maze.path_not_empty():
                    self._scan_position()
                    self.maze.print_maze()
    
                    next_path = self.maze.get_next_move()
    
                    if DEBUG_LOGGING:
                        print(f'Mouse position: {self.maze.mouse_position}')
                        print(f'Finish: {self.maze.finish_cell}')
                        print(f'Next move: {next_path}')
                        print(f'Path: {self.maze.path}')
                    
                    if next_path == 'F':
                        if self.maze.get_wall(Directions.up):
                            recalculate = True
                        else:
                            self._move_by_direction(direction=self.maze.mouse_direction)
                    elif next_path == 'R':
                        if self.maze.get_wall(Directions.right):
                            recalculate = True
                        else:
                            self._move_by_direction(direction=self.maze.mouse_direction + Directions.right)
                            self.maze.get_next_move() # pop F after turn
                    elif next_path == 'A':
                        self._move_by_direction(direction=self.maze.mouse_direction + Directions.down)
                        self.maze.get_next_move() # pop F after turn
                    elif next_path == 'L':
                        if self.maze.get_wall(Directions.left):
                            recalculate = True
                        else:
                            self._move_by_direction(direction=self.maze.mouse_direction + Directions.left)
                            self.maze.get_next_move() # pop F after turn
                    
                    if recalculate:
                        self.maze.floodfill()
                        path_exists = self.maze.find_path(self.maze.mouse_position)
                        recalculate = False
                        self.maze.print_maze()
                        if DEBUG_LOGGING:
                            print("Recalculated!")
                        break
        
        self._scan_position()
        return path_exists
        
                
    def reset_position(self):
        self.maze.reset_position()
    
    def post_task_1(self):
        return self.maze.post_result()

In [98]:
solver = Solver(my_token, manual_control=True)
solver.reset_position()

In [99]:
solver.shortest()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| 14  13  12  11  10  9   8   7   8   9   10  11  12  13  14  15|
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 13  12  11  10  9   8   7   6   7   8   9   10  11  12  13  14|
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 12  11  10  9   8   7   6   5   6   7   8   9   10  11  12  13|
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 11  10  9   8   7   6   5   4   5   6   7   8   9   10  11  12|
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 10  9   8   7   6   5   4   3   4   5   6   7   8   9   10  11|
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 9   8   7   6   5   4   3   2   3   4   5   6   7   8   9   10|
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 8   7   6   5   4   3   2   1   2   3   4   5   6   7   8   9 |
+   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +   +
| 7   6   

KeyboardInterrupt: 

In [232]:
solver.reset_position()
solver.maze.lock_maze()
solver.shortest()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 65  64  63| 56  55  54  53  52| 0 | 0 | 0 |
+---+---+---+---+---+---+   +   +   +---+---+---+   +---+---+---+
| 0 | 0 | 0 | 92  91  90| 65| 62| 57  58| 49  50  51| 0 | 0 | 0 |
+---+---+---+   +---+   +   +   +---+   +   +---+---+---+---+---+
| 0 | 0 | 0 | 93| 88  89| 66| 61  60  59| 48  47  46  45| 0 | 0 |
+---+---+---+   +   +---+   +---+---+---+---+---+---+   +---+---+
| 0 | 0 | 0 | 94| 87  86| 67  68  69| 72  73| 0 | 43  44| 0 | 0 |
+---+---+---+   +---+   +---+---+   +   +   +---+   +---+---+---+
| 0 | 0 | 96  95| 0 | 85| 82  81| 70  71| 74| 39| 42  41| 0 | 0 |
+---+---+   +---+---+   +   +   +   +---+   +   +---+   +---+---+
| 0 | 0 | 97| 0 | 0 | 84  83| 80| 71| 76  75| 38  39  40| 0 | 0 |
+---+---+   +---+---+---+---+   +---+   +---+   +---+---+---+---+
|100  99  98| 0 | 6   5   4 | 79  78  77| 36| 37| 0 | 0 | 0 | 0 |
+   +---+---+---+---+   +   +---+---+---+   +   +---+---+---+---+
|101| 0 | 

True

In [102]:
mouse = Mouse(my_token, True)

In [103]:
while True:
    mouse.update_sensor_data()
    print(mouse)

< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y: -197.96
error: -24.24
< 89.3 True 
^ 322.11 False 
> 288.7 False 
angle: 92 (92)
x: 83.42 y:

KeyboardInterrupt: 

In [61]:
mouse.around()